In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 14906, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 14906 (delta 11), reused 12 (delta 2), pack-reused 14880
Receiving objects: 100% (14906/14906), 13.93 MiB | 9.35 MiB/s, done.
Resolving deltas: 100% (10249/10249), done.
/content/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 1.6 MB 36.1 MB/s 
Setup complete. Using torch 1.13.0+cu116 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="TBSZIcJvZGLN6tqEEfeO")
project = rf.workspace("tfnet-night-vision").project("mul")
version = project.version(4)
dataset = version.download("yolov5")

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 957 kB/s 
     |████████████████████████████████| 138 kB 63.4 MB/s 
     |████████████████████████████████| 145 kB 73.5 MB/s 
     |████████████████████████████████| 67 kB 7.0 MB/s 
     |████████████████████████████████| 54 kB 3.8 MB/s 
     |████████████████████████████████| 178 kB 83.1 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=b443a349c180eb7fa0c5402665d5d86cd789af914134aae61572a8d1b068d6dc
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: certifi
    Found existing ins

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Mul-2 in yolov5pytorch:: 100%|██████████| 16733/16733 [00:03<00:00, 4225.85it/s]


names:
- Bird
- Drone
- Helicopter
- Plane
nc: 4
roboflow:
  license: CC BY 4.0
  project: mul
  url: https://universe.roboflow.com/tfnet-night-vision/mul/dataset/2
  version: 2
  workspace: tfnet-night-vision
test: ../test/images
train: Mul-2/train/images
val: Mul-2/valid/images


In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial
%cat //content/yolov5/models/hub/yolov5-p34.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors: 3  # AutoAnchor evolves 3 anchors per P output layer

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [ [ -1, 1, Conv, [ 64, 6, 2, 2 ] ],  # 0-P1/2
    [ -1, 1, Conv, [ 128, 3, 2 ] ],  # 1-P2/4
    [ -1, 3, C3, [ 128 ] ],
    [ -1, 1, Conv, [ 256, 3, 2 ] ],  # 3-P3/8
    [ -1, 6, C3, [ 256 ] ],
    [ -1, 1, Conv, [ 512, 3, 2 ] ],  # 5-P4/16
    [ -1, 9, C3, [ 512 ] ],
    [ -1, 1, Conv, [ 1024, 3, 2 ] ],  # 7-P5/32
    [ -1, 3, C3, [ 1024 ] ],
    [ -1, 1, SPPF, [ 1024, 5 ] ],  # 9
  ]

# YOLOv5 v6.0 head with (P3, P4) outputs
head:
  [ [ -1, 1, Conv, [ 512, 1, 1 ] ],
    [ -1, 1, nn.Upsample, [ None, 2, 'nearest' ] ],
    [ [ -1, 6 ], 1, Concat, [ 1 ] ],  # cat backbone P4
    [ -1, 3, C3, [ 512, False ] ],  # 13

    [ -1, 1, Conv, [ 256, 1, 1 ] ],
    [ -1, 1, nn.Upsample, [ None,

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/YOLOG-p345.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors: 2  # AutoAnchor evolves 3 anchors per P output layer

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [ [ -1, 1, Conv, [ 64, 6, 2, 2 ] ],  # 0-P1/2
    [ -1, 1, GhostConv, [ 128, 3, 2 ] ],  # 1-P2/4
    [ -1, 3, C3Ghost, [ 128 ] ],
    [ -1, 1, GhostConv, [ 256, 3, 2 ] ],  # 3-P3/8
    [ -1, 6, C3Ghost, [ 256 ] ],
    [ -1, 1, GhostConv, [ 512, 3, 2 ] ],  # 5-P4/16
    [ -1, 9, C3Ghost, [ 512 ] ],
    [ -1, 1, GhostConv, [ 1024, 3, 2 ] ],  # 7-P5/32
    [ -1, 3, C3Ghost, [ 1024 ] ],
    [ -1, 1, SPPF, [ 1024, 5 ] ],  # 9
  ]
# YOLOv5 v6.0 head with (P3, P4) outputs
head:
  [ [ -1, 1, GhostConv, [ 512, 5, 1 ] ],
    [ -1, 1, nn.Upsample, [ None, 2, 'nearest' ] ],
    [ [ -1, 6 ], 1, Concat, [ 1 ] ],  # cat backbone P4
    [ -1, 3, C3Ghost, [ 512, False ] ],  # 13

    [ -1, 1, GhostConv, [ 256, 5, 1 ] ],
    [ -1, 1, nn.Upsample, [ None, 2, 'nearest' ] ],
    [ [ -1, 4 ], 1, Concat, [ 1 ] ],  # cat backbone P3
    [ -1, 3, C3Ghost, [ 256, False ] ],  # 17 (P3/8-small)

    [ -1, 1, GhostConv, [ 256, 5, 2 ] ],
    [ [ -1, 14 ], 1, Concat, [ 1 ] ],  # cat head P4
    [ -1, 3, C3Ghost, [ 512, False ] ],  # 20 (P4/16-medium)

    [ [ 17, 20 ], 1, Detect, [ nc, anchors ] ],  # Detect(P3, P4)
  ]

In [ ]:
## time its performance
%%time
%cd /content/yolov5/
!python train.py --img 320 --batch-size -1 --epochs 600 --optimizer AdamW --data {dataset.location}/data.yaml --cfg ./models/YOLOG-p345.yaml --hyp data/hyps/hyp.scratch-high.yaml --weights '' --project "/content/gdrive/MyDrive/Runs" --name YOLOG-p345_results  --cache

/content/yolov5
train: weights=, cfg=./models/YOLOG-p345.yaml, data=/content/yolov5/Mul-2/data.yaml, hyp=data/hyps/hyp.scratch-high.yaml, epochs=600, batch_size=-1, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=AdamW, sync_bn=False, workers=8, project=/content/gdrive/MyDrive/Runs, name=YOLOG-p345_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 417 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.

In [ ]:
#Later, if it is stopped prematurely and I need to resume, I use
#%%time
#%cd /content/yolov5/
#!python train.py --img 320--data {dataset.location}/data.yaml --hyp data/hyps/hyp.scratch-high.yaml --cfg ./models/YOLOG-p345.yaml --weights '' --project "/content/gdrive/MyDrive/Runs"  --name YOLOG-p345_results --cache  --resume "/content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/last.pt"

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason...
#from utils.plots import plot_results  # plot results.txt as results.png
#Image(filename='/content/yolov5/runs/train/YOLOG-p34_results/results.png', width=1000)  # view results.png

In [ ]:
#!python val.py --task study --weights runs/train/YOLOG-p34_results/weights/best.pt--data {dataset.location}/data.yaml --iou 0.5

In [ ]:
#!python val.py --task speed --weights runs/train/YOLOG-p34_results/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5

In [ ]:
#%cd /content/yolov5/
#!python detect.py --weights runs/train/YOLOG-p34_results/weights/best.pt --img 320 --conf 0.4 --source /content/yolov5/Mul-1/valid/images
##--visualize

In [ ]:
#!python val.py --weights --img 320 --weights runs/train/YOLOG-p34_results/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 --verbose --save-txt

In [ ]:
!python val.py --task study --weights /content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/last.pt --data {dataset.location}/data.yaml --iou 0.5

In [ ]:
!python val.py --task speed --weights /content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/last.pt --data {dataset.location}/data.yaml --iou 0.5

In [ ]:
!python val.py --img 320 --weights /content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/last.pt --data {dataset.location}/data.yaml --iou 0.5 --verbose

In [ ]:
%cd /content/yolov5/
!python detect.py --weights /content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/last.pt --img 320 --conf 0.4 --source /content/yolov5/Mul-1/valid/images
##--visualize

In [ ]:
##I have created a zip file:
!zip -r /content/YOLOG-p345.zip /content/gdrive/MyDrive/Runs/YOLOG-p34_results
!zip -r /content/YOLOG-p345.zip /content/yolov5/runs/
##Than I have downloded that zip file:
from google.colab import files
files.download("/content/YOLOG-p345.zip")

In [ ]:
#### Visulize each layers feature map
#######Run this cell to know the whats inside every lare feature
#### stop it ur self after one image is done
## Print whole folder feature maps and save them
#!python detect.py --weights /content/yolov5/mfnets.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images --visualize
####### Only single image
#!python detect.py --weights runs/train/GAANet-p2345_results/weights/best.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77.jpg --visualize